# Import dependencies

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from time import sleep
import requests
from google.colab import files
from bs4 import BeautifulSoup

# Scraping

To begin with, let's take uhmmm... Mc Donald's

In [2]:
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

users = []
userReviewNum = []
ratings = []
locations = []
dates = []
reviews = []

from_page = 1
to_page = 6
company = 'mc-donalds.com'

for i in range(from_page, to_page+1):

   result = requests.get(fr"https://www.trustpilot.com/review/{company}?page={i}")
   soup = BeautifulSoup(result.content)

   # Trust Pilot was setup in a way that's not friendly to scraping, so this hacky method will do.
   soup2list(soup.find_all('span', {'class','typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
   soup2list(soup.find_all('div', {'class','typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
   soup2list(soup.find_all('span', {'class','typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l'}), userReviewNum)
   soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), dates)
   soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
   soup2list(soup.find_all('div', {'class','styles_reviewContent__0Q2Tg'}), reviews)

   # To avoid throttling
   sleep(1)

review_data = pd.DataFrame(
{
   'Username':users,
   'Total reviews':userReviewNum,
   'location':locations,
   'date':dates,
   'content':reviews,
   'Rating': ratings
})

In [3]:
# create a dataframe for reviews
review_content = review_data['content']
df = pd.DataFrame(data = {'review': review_content})

df

,review
0,Couldn't get an order right to save their life...
1,McDonald's - Five Lane Ends I never seem to le...
2,If i could give you 0 stars i wouldIf i could ...
3,The order I placed from McDonald's…The order I...
4,Shocking place to get foodShocking place to ge...
...,...
115,"So badSo bad, there are better optionsDate of ..."
116,Fast food fastFast food fast. I like that the...
117,I love Mac DonaldsThe food is amazing and ther...
118,Poisoning America and the WorldThis company ha...


In [4]:
# examine review
df['review'][0]

"Couldn't get an order right to save their life It's one thing when you're in store and can confront the staff about them giving you incomplete orders, but I shouldn't be surprised when I ordered a bic mac for delivery and there was no burger! After being at work ALL DAY, I spent £12 to recieve only a shake and fries! I tried calling several times but got not answer.  Could have got chips from the local chippy for a couple quid. What an absolute waste.Date of experience: March 11, 2024"

In [5]:
df['review'][2]

"If i could give you 0 stars i wouldIf i could give you 0 stars i would. You are so embarrassing for supporting genocide. IMAGINE. couldn't be me. you are disgusting you fagg autistic c u n t. I hope you kill your self no one likes you. go cut yourself, hang yourself, jump off a building. commit. I HOPE YOUR PLACE BURNS DOWN AND YOUR PEOPLE ARE STILL IN IT. IMAGINE SUPPORTING A FAKE COUNTRY YOU FAKE A S S    H O E S. I mean I understand since your meat is fake.Date of experience: February 07, 2024"

Wow this guy ☝️ was livid.


Anyway, it seems like we have a problem. At the end of every review, an unwelcome 'Date of experience' is included.

In [10]:
import re

def remove_date(review):
    # Define the date pattern using regular expression
    date_pattern = r"Date of experience: \w+ \d{1,2}, \d{4}$"
    # Replace the date pattern with an empty string
    cleaned_review = re.sub(date_pattern, "", review)
    return cleaned_review.strip()  # Strip any leading/trailing whitespace

# Apply the function to each review
cleaned_reviews = [remove_date(review) for review in reviews]

df['review'] = cleaned_reviews

In [11]:
df['review'][0]

"Couldn't get an order right to save their life It's one thing when you're in store and can confront the staff about them giving you incomplete orders, but I shouldn't be surprised when I ordered a bic mac for delivery and there was no burger! After being at work ALL DAY, I spent £12 to recieve only a shake and fries! I tried calling several times but got not answer.  Could have got chips from the local chippy for a couple quid. What an absolute waste."

In [12]:
df['review'][5]

"Untrained staff rude behaviour MC…Untrained staff rude behaviour MC Donald's refused to take order after 9:40 pm.pre ordered food is cancelled by the staff without acknowledged the staff doesn't have any manners and doesn't care about how to deal with a cancelled order they didn't even tried to call back the customer and let us know the order cancellation details the staff who talked from the outlet ( after we contacted them) is not polite and doesn't offer an order replacement options or nothing Completely unsatisfied with the staff in Abad nucleus mall McDonald's outlet kochi.Terrible experience..becoz the staff seemed untrained they were slow made mistake with my order and seemed confused wen I asked for help.frustating staff untrained and rude."

In [13]:
df['review'][2]

"If i could give you 0 stars i wouldIf i could give you 0 stars i would. You are so embarrassing for supporting genocide. IMAGINE. couldn't be me. you are disgusting you fagg autistic c u n t. I hope you kill your self no one likes you. go cut yourself, hang yourself, jump off a building. commit. I HOPE YOUR PLACE BURNS DOWN AND YOUR PEOPLE ARE STILL IN IT. IMAGINE SUPPORTING A FAKE COUNTRY YOU FAKE A S S    H O E S. I mean I understand since your meat is fake."


# Building Model

In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import sequence
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

In [15]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [16]:
# function for rating review

def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors = 'pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1


df['rating'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [17]:
df

,review,rating
0,Couldn't get an order right to save their life...,1
1,McDonald's - Five Lane Ends I never seem to le...,1
2,If i could give you 0 stars i wouldIf i could ...,1
3,The order I placed from McDonald's…The order I...,1
4,Shocking place to get foodShocking place to ge...,1
...,...,...
115,"So badSo bad, there are better options",1
116,Fast food fastFast food fast. I like that the...,4
117,I love Mac DonaldsThe food is amazing and ther...,5
118,Poisoning America and the WorldThis company ha...,1


In [18]:
# unique ratings
df['rating'].unique().tolist()

[1, 4, 2, 5, 3]

In [19]:
average_rating = df['rating'].mean()
print(f'Average rating is {round(average_rating, 1)} out of 5 stars.')

Average rating is 1.9 out of 5 stars.


# Making it a function

In [20]:
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors = 'pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1


def review(company_link):
  users = []
  userReviewNum = []
  ratings = []
  locations = []
  dates = []
  reviews = []

  from_page = 1
  to_page = 6


  for i in range(from_page, to_page+1):

    result = requests.get(fr"https://www.trustpilot.com/review/{company_link}?page={i}")
    soup = BeautifulSoup(result.content)

    # Trust Pilot was setup in a way that's not friendly to scraping, so this hacky method will do.
    soup2list(soup.find_all('span', {'class','typography_heading-xxs__QKBS8 typography_appearance-default__AAY17'}), users)
    soup2list(soup.find_all('div', {'class','typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_detailsIcon__Fo_ua'}), locations)
    soup2list(soup.find_all('span', {'class','typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l'}), userReviewNum)
    soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), dates)
    soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')
    soup2list(soup.find_all('div', {'class','styles_reviewContent__0Q2Tg'}), reviews)

    # To avoid throttling
    sleep(1)

  review_data = pd.DataFrame(
  {
    'Username':users,
    'Total reviews':userReviewNum,
    'location':locations,
    'date':dates,
    'content':reviews,
    'Rating': ratings
  })

  review_content = review_data['content']
  df = pd.DataFrame(data = {'review': review_content})

  tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

  model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')



  df['rating'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

  average_rating = df['rating'].mean()
  print(f'Average rating is {round(average_rating, 1)} out of 5 stars.')

# Review companies




*    Old Mutual Invest





In [21]:
company_link = 'www.oldmutualinvest.com'

review(company_link)

Average rating is 1.7 out of 5 stars.




*   Capitec Bank




In [22]:
company_link = 'capitecbank.co.za'

review(company_link)

Average rating is 1.3 out of 5 stars.




*   Discovery



In [23]:
company_link = 'discovery.co.za'

review(company_link)

Average rating is 1.0 out of 5 stars.




*   Nedbank



In [24]:
company_link = 'nedbank.co.za'

review(company_link)

Average rating is 1.2 out of 5 stars.


*   Makro



In [25]:
company_link = 'makro.co.za'

review(company_link)

Average rating is 1.2 out of 5 stars.
